In [2]:
from tensorflow.keras.datasets import cifar10
import numpy as np
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
# 샘플링 비율 설정 (10%)
sampling_ratio = 0.1

# 훈련 데이터 샘플링
num_samples = int(len(X_train) * sampling_ratio)
indices = np.random.choice(len(X_train), num_samples, replace=False) # 비복원추출
X_train_sampled = X_train[indices]
y_train_sampled = y_train[indices]

In [4]:
# 테스트 데이터 샘플링
num_samples_test = int(len(X_test) * sampling_ratio)
indices_test = np.random.choice(len(X_test), num_samples_test, replace=False)
X_test_sampled = X_test[indices_test]
y_test_sampled = y_test[indices_test]
print("새로운 훈련 데이터 크기:", X_train_sampled.shape)
print("새로운 테스트 데이터 크기:", X_test_sampled.shape)

새로운 훈련 데이터 크기: (5000, 32, 32, 3)
새로운 테스트 데이터 크기: (1000, 32, 32, 3)


In [5]:
import tensorflow as tf
X_train_resized = tf.image.resize(X_train_sampled, [224, 224])
X_test_resized = tf.image.resize(X_test_sampled, [224, 224])

In [6]:
train_images = X_train_resized / 255.0
test_images = X_test_resized / 255.0

print(train_images.shape)
print(test_images.shape)

(5000, 224, 224, 3)
(1000, 224, 224, 3)


In [7]:
# 종속변수 확인
print(type(y_test_sampled))
print(y_test_sampled[:10])

# 변수명 변경
train_labels = y_train_sampled
test_labels = y_test_sampled

<class 'numpy.ndarray'>
[[9]
 [1]
 [1]
 [6]
 [6]
 [4]
 [8]
 [8]
 [6]
 [8]]


In [8]:
from keras import models
from keras import layers
from tensorflow.keras.regularizers import l2

model = models.Sequential()
weight_decay = 0.0001

# Block 1
model.add(layers.Conv2D(64, (3, 3), padding='same',input_shape=(224, 224, 3),kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(64, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=l2(weight_decay),activation='relu'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

In [9]:
# 완전연결층
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5)) # Dropout
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5)) # Dropout
model.add(layers.Dense(10, activation='softmax')) # 본래 1000 분류이나, CIFAR-10의 10분류로 수정

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [ ]:
# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

# 모델 훈련
# 데이터와 훈련량이 충분하지 않기 때문에 성능은 좋지 않다
history = model.fit(train_images, train_labels, epochs=10, batch_size=128,validation_data=(test_images, test_labels))

In [ ]:
# 데이터와 훈련량이 충분하지 않기 때문에 성능은 좋지 않다
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('Accuracy of each epoch:', acc)
epochs = range(1, len(acc) +1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
# 랜덤 한 개 이미지 추출
import numpy as np
sample = np.random.choice(np.arange(0, len(test_images)))
print(sample)
print(test_labels[sample])

In [ ]:
# 모델을 사용해 예측
predictions = model.predict(test_images[sample:sample+1])

# 가장 큰 확률의 인덱스
predicted_class = np.argmax(predictions, axis=1)[0]
print("예측된 숫자:", predicted_class)

# 클래스별 확률
print("\n클래스별 확률:")
for i, prob in enumerate(predictions[0]):
  print(f"Class {i}: {prob:.4f}")